TREINAMENTO INTELIGENCIA ARTIFICAL


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
import smtplib
from email.mime.image import MIMEImage
from flask import Flask, jsonify, request, send_file
from flask_cors import CORS


# Previsão de preços
def previsao_precos(data, email, ativo, decisao_email):
    periodo = treinamento(ativo)[:data]
    data_inicial = datetime.now()
    datas = [data_inicial + timedelta(days=i) for i in range(len(periodo))]
    plt.plot(datas, periodo, marker='o',
             linestyle='-', color='b', label='Valores')
    plt.gca().xaxis.set_major_formatter(
        plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
    plt.gcf().autofmt_xdate()
    imagem_path = f'temp/{ativo}.png'
    plt.ylabel('Valores')
    plt.title('Gráfico de Valores ao Longo do Tempo')
    plt.legend()
    plt.savefig(imagem_path)
    plt.close()
    if decisao_email == "Sim":
        enviar_Email(email, ativo)
    elif decisao_email == "Não":
        pass

# ENVIAR GRAFICO POR EMAIL
def enviar_Email(destinatario,ativo):
    smtp_server = 'smtp-mail.outlook.com'
    smtp_port = 587
    smtp_username = 'financ.iaInveste@hotmail.com'
    smtp_password = 'alancruz123@'

    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = destinatario
    msg['Subject'] = 'Gráfico enviado por e-mail'

    with open(f'temp/{ativo}.png', 'rb') as anexo:
        imagem_anexo = MIMEImage(anexo.read(), name=f'{ativo}.png')
        msg.attach(imagem_anexo)

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.sendmail(smtp_username, destinatario, msg.as_string())
        server.quit()
        print('E-mail enviado com sucesso!')
    except Exception as e:
        print('Ocorreu um erro:', str(e))


# MELHOR DATA DE VENDA COM BASE NO PERIODO
def calcular_data_de_venda_futura(ativo):

    index_max = int(np.argmax(treinamento(ativo)))
    date_start = datetime.now()
    date_max = date_start + timedelta(days=index_max)
    max_value = treinamento(ativo)[index_max]

    content = {
        "date_max": date_max.strftime("%d-%m-%Y"),
        "max_value": f"R$ {max_value}"
    }

    return content
    

# MELHOR DATA DE COMPRA E VENDA
def melhor_data_compra_venda(valor_medio_compra, ativo, periodo_dias=30):
    datas_futuras = [datetime.now() + timedelta(days=i)
                     for i in range(1, periodo_dias + 1)]
    previsoes = []

    for i, data_futura in enumerate(datas_futuras):
        if i < len(treinamento(ativo)):
            preco_previsto = treinamento(ativo)[i]
            previsoes.append(
                (data_futura.strftime('%Y-%m-%d'), preco_previsto))

    melhores_datas_compra = [data for data,
                             preco in previsoes if preco < valor_medio_compra]
    melhores_datas_venda = [data for data,
                            preco in previsoes if preco > valor_medio_compra]

    if melhores_datas_compra:
        melhor_data_compra = min(melhores_datas_compra)
        preco_melhor_data_compra = next(
            preco for data, preco in previsoes if data == melhor_data_compra)
        melhor_data_compra_formatada = datetime.strptime(melhor_data_compra, "%Y-%m-%d").strftime("%d-%m-%Y")
        best_buy = f"Melhor data de compra: {melhor_data_compra_formatada} (preço: R$ {preco_melhor_data_compra:.2f}, abaixo da média)"
    else:
        best_buy = "Não há oportunidade de compra abaixo da média no período especificado."

    if melhores_datas_venda:
        melhor_data_venda = min(melhores_datas_venda)
        preco_melhor_data_venda = next(
            preco for data, preco in previsoes if data == melhor_data_venda)
        melhor_data_venda_formatada = datetime.strptime(melhor_data_venda, "%Y-%m-%d").strftime("%d-%m-%Y")
        best_sell = f"Melhor data de venda: {melhor_data_venda_formatada} (preço: R$ {preco_melhor_data_venda:.2f}, acima da média)"
    else:
        best_sell = "Não há oportunidade de venda acima da média no período especificado."
        
    content = {
        "best_buy_date": best_buy,
        "best_sell_date": best_sell
    }
    return content

# TREINAMENTO DO ALGORITMO
def treinamento(nome):

    url = f"https://raw.githubusercontent.com/Alan-Figueiredo/projeto_preco_acoes/main/datasets/{nome}.csv"
    dataset = pd.read_csv(url)
    dataset = dataset.drop("Adj Close", axis=1)

    pd.options.display.float_format = '{:.2f}'.format

    dataset['Date'] = pd.to_datetime(dataset['Date'])
    dataset['DaysFromReference'] = (
        dataset['Date'] - dataset['Date'].min()).dt.days + 1
    dataset = dataset.fillna(0)
    target_column = "Close"
    X = dataset.drop([target_column, "Date"], axis=1)
    y = dataset[target_column]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    k = 25
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)

    predicion = knn.predict(X_test_scaled)
    predicion = np.round(predicion, 2)
  
    return predicion


app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


@app.route("/api/previsao", methods=["POST"])
def previsão():
    dados = request.json
    
    dataFull = dados.get("data_select")
    email = dados.get("email_select")
    nomeAtivo = dados.get("ativo_select")
    decisao_email = dados.get("email_desicion")
    data_inicial = datetime.now()
    data_final = datetime.strptime(dataFull, '%Y-%m-%d')
    data = (data_final - data_inicial).days

    previsao = previsao_precos(data, email, nomeAtivo, decisao_email)

    return jsonify(previsao)

@app.route("/api/venda_futura", methods=["POST"])
def venda_futura():
    dados = request.json
    
    nomeAtivo = dados.get("ativo_select")
    vendaFtr = calcular_data_de_venda_futura(nomeAtivo)
    return jsonify(vendaFtr)

@app.route("/api/compra_venda", methods=["POST"])
def compra_venda():
    dados = request.json

    nomeAtivo = dados.get("ativo_select")
    precoMedio = int(dados.get("precoMed_input"))
    dias = int(dados.get("dias_select"))
    compraVenda = melhor_data_compra_venda(precoMedio, nomeAtivo, dias)
    return jsonify(compraVenda)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Nov/2023 13:22:55] "OPTIONS /api/compra_e_venda HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2023 13:23:37] "OPTIONS /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:42] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:42] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:42] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:42] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:45] "OPTIONS /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:46] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:46] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:51] "POST /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:51] "OPTIONS /api/venda_futura HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2023 13:23:53] "POST /api/venda_futura HTTP/1.1" 200 -
127.0